# Two Heap

In many problems, where we are given a set of elements such that we can divide them into two parts. To solve the problem, we are interested in knowing the smallest element in one part and the biggest element in the other part. This pattern is an efficient approach to solve such problems.

This pattern uses two Heaps to solve these problems; A Min Heap to find the smallest element and a Max Heap to find the biggest element.

## Find the Median of a Number Stream (medium)

Design a class to calculate the median of a number stream. The class should have the following two methods:

1. insertNum(int num): stores the number in the class
2. findMedian(): returns the median of all numbers inserted in the class


If the count of numbers inserted in the class is even, the median will be the average of the middle two numbers.

###  Brute-Force Sorting O(NlogN) Find Median O(1)

### Two Heap Insert O(logN) Find Median O(1)

In [7]:
from heapq import *

class Stream():
    
    maxHeap = []
    minHeap = []
    
    def insert_num(self, num):
        if not self.maxHeap or -self.maxHeap[0] >= num:
            heappush(self.maxHeap, -num)
        else:
            heappush(self.minHeap, num)

        if len(self.maxHeap) > len(self.minHeap) + 1:
            heappush(self.minHeap, -heappop(self.maxHeap))
        elif len(self.maxHeap) < len(self.minHeap):
            heappush(self.maxHeap, -heappop(self.minHeap))
            
    def find_median(self):
        if len(self.maxHeap) == len(self.minHeap):
            return -self.maxHeap[0] / 2.0 + self.minHeap[0] / 2.0
        
        return -self.maxHeap[0]
    

In [8]:
s = Stream()
s.insert_num(3)
s.insert_num(1)
s.find_median()

2.0

In [9]:
stream = Stream()
stream.insert_num(3)
stream.insert_num(1)
print(stream.find_median())
stream.insert_num(5)
print(stream.find_median())
stream.insert_num(4)
print(stream.find_median())

2.0
3
3.0


## Sliding Window Median (hard)

Given an array of numbers and a number ‘k’, find the median of all the ‘k’ sized sub-arrays (or windows) of the array.

### Sort O(KlogK) Total O(NKlogK)

In [4]:
def solution(arr, k):
    
    result = []
    n = len(arr)
    
    if k % 2 == 0:
        is_odd = False
    else:
        is_odd = True
    
    for i in range(n-k+1):
        subarr = arr[i:i+k]
        subarr.sort()
        if is_odd:
            result.append(subarr[k//2])
        else:
            result.append((subarr[k//2-1]+subarr[k//2])/2.0)
        
    
    return result
    

### Insert O(logK) Remove O(K) Total O(N(K+logK))

In [5]:
from heapq import *
import heapq

class Solution():
    
    def __init__(self):
        self.maxHeap = []
        self.minHeap = []
    
    def rebalance_heaps(self):
        if len(self.maxHeap) > len(self.minHeap) + 1:
            heappush(self.minHeap, -heappop(self.maxHeap))
        
        elif len(self.maxHeap) < len(self.minHeap):
            heappush(self.maxHeap, -heappop(self.minHeap))
    
    def remove(self, element):
        
        if element <= -self.maxHeap[0]:
            heap = self.maxHeap
            element = -element
        else:
            heap = self.minHeap
        
        ind = heap.index(element)
        # swap
        heap[ind] = heap[-1]
        del heap[-1]
        
        # fix up
        if ind < len(heap):
            heapq._siftup(heap, ind)
            heapq._siftdown(heap, 0, ind)
        
        self.rebalance_heaps()
        
    
    def insert(self, element):
        if not self.maxHeap or element <= -self.maxHeap[0]:
            heappush(self.maxHeap, -element)
        else:
            heappush(self.minHeap, element)
            
        self.rebalance_heaps()
    
    def median(self):
        if len(self.maxHeap) == len(self.minHeap):
            return -self.maxHeap[0] / 2.0 + self.minHeap[0] / 2.0
        
        return -self.maxHeap[0]
    
    
    def find_median(self, nums, k):
        result = []
        n = len(nums)
        
        for i in range(k):
            self.insert(nums[i])
            
        result.append(self.median())
        
        for i in range(k,n):
            
            self.remove(nums[i-k])
            self.insert(nums[i])
            result.append(self.median())
        
        return result
        


In [6]:
arr = [1, 2, -1, 3, 5]
sol = Solution()
sol.find_median(arr, 3)

[1, 2, 3]

## Maximize Capital (hard)

Given a set of investment projects with their respective profits, we need to find the most profitable projects. We are given an initial capital and are allowed to invest only in a fixed number of projects. Our goal is to choose projects that give us the maximum profit. Write a function that returns the maximum total capital after selecting the most profitable projects.

### Recursive + Heap, We don't know if the capital and profits are sorted or not.

In [4]:
from heapq import heappop, heappush

def find_maximum_capital(capital, profits, numberOfProjects, initialCapital):
    
    if numberOfProjects == 0:
        return 0
    
    affordable = []
    for i in range(len(capital)):
        if capital[i] <= initialCapital:
            heappush(affordable, -profits[i])
    
    return -affordable[0] + find_maximum_capital(capital, profits, numberOfProjects-1, initialCapital-affordable[0])
        

In [6]:
find_maximum_capital([0, 1, 2], [1, 2, 3], 2, 1) + 1

6

## Next Interval (hard)

Given an array of intervals, find the next interval of each interval. In a list of intervals, for an interval ‘i’ its next interval ‘j’ will have the smallest ‘start’ greater than or equal to the ‘end’ of ‘i’.

Write a function to return an array containing indices of the next interval of each input interval. If there is no next interval of a given interval, return -1. It is given that none of the intervals have the same start point.

## Brute Force O(N<sup>2</sup>)

Very Ugly

In [11]:
class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
def find_next_interval(intervals):
    result = [-1] * len(intervals)
    
    n = len(intervals)
    for i in range(n):
        end = intervals[i].end
        min_start = None
        for j in range(n):
            if intervals[j].start >= end:
                if min_start is None:
                    min_start = intervals[j].start
                    result[i] = j
                else:
                    if min_start > intervals[j].start:
                        result[i] = j

    return result

## Two Heap O(NlogN)

Using two maximum heaps to store both the start and the end of intervals. 

First step: O(NlogN)
Second step: O(NlogN)

In [7]:
from heapq import heappop, heappush

class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end


def find_next_interval(intervals):
    result = [-1] * len(intervals)
    
    # max start and end
    heap1 = []
    heap2 = []
    
    for i in range(len(intervals)):
        heappush(heap1, (-intervals[i].start, i))
        heappush(heap2, (-intervals[i].end, i))
    
    
    _, idx = heappop(heap2)
    result[idx] = -1
    
    while heap2:
        end, idx = heappop(heap2)
        idx2 = -1
        start = None
        while -heap1[0][0] >= -end:
            start, idx2 = heappop(heap1)
        result[idx] = idx2
        
        # push back !
        if start:
            heappush(heap1, (start, idx2))
        
    return result
    

In [10]:
result = find_next_interval([Interval(3, 4), Interval(1, 5), Interval(4, 6), Interval(5, 8)])
print(result)

[2, 3, -1, -1]
